In [41]:
from kfto_sdk_mnist import train_func_2
from kubeflow.training import TrainingClient
from kubernetes import client
import time

In [ ]:
# parameters
num_gpus = "${num_gpus}"
train_function = "${train_function}"
openshift_api_url = "${api_url}"
namespace = "${namespace}"
token = "${token}"
password =  "${password}"

In [ ]:
config = client.Configuration(host=openshift_api_url, api_key=token)
config.verify_ssl = False
client = TrainingClient(client_configuration=config)

In [44]:

client.create_job(
   name="pytorch-ddp",
   namespace=namespace,
   train_func=train_function,
   num_workers=2,
   resources_per_worker={"gpu": num_gpus},
)

AttributeError: module 'kubernetes.client' has no attribute 'create_job'

In [28]:
while not client.is_job_succeeded(name="pytorch-ddp", namespace=namespace) {
    time.sleep(1)
}

/home/kpostoffice/.pyenv/versions/kfto-testing/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.ods-qe-psi-05.osp.rh-ods.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [40]:
TrainingClient().delete_job(
    name="pytorch-ddp",
    namespace=namespace,
)

/home/kpostoffice/.pyenv/versions/kfto-testing/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.ods-qe-psi-05.osp.rh-ods.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


True